In [12]:
  '''
  PUSING MY COLLAB PROJECT TO GITHUB
  '''
# !ssh-keygen -t rsa -b 4096 -C "fabianbernard49@gmail.com" -f ~/.ssh/id_rsa -N ""  #Create ssh pair key
# !cat ~/.ssh/id_rsa.pub #Print out your
!ssh-keyscan -H github.com >> ~/.ssh/known_hosts #Make github a knownhost
!ssh -T git@github.com #Aunthenticate github

  # Verify your identity
!git config --global user.name "coded-by-49"
!git config --global user.email "fabianbernard49@gmail.com"

# %cd /content/drive/MyDrive/Git_repos #Create a folder for git hub repos if its your first time
# %cd /content/drive/MyDrive/Git_repos/Telecom-churn-model/ #Enter that folder
# !git clone git@github.com:coded-by-49/Telecom-churn-model.git  #Clone your repo into it

#Copy your project note-book progress into it and open the directory were your copy is stored
!cp "/content/drive/MyDrive/Colab Notebooks/Tele_churn.ipynb" /content/drive/MyDrive/Git_repos/Telecom-churn-model/ #Copy your project note-book progress into it
%cd /content/drive/MyDrive/Git_repos/Telecom-churn-model # open the directory were your copy is stored
!git status
!git add Tele_churn.ipynb #Add the file itself to avoid json file upload in repo
!git commit -m "Optimisation of Model precision and recall using feature engineering"
!git push origin main

# github.com:22 SSH-2.0-1581bab2
# github.com:22 SSH-2.0-1581bab2
# github.com:22 SSH-2.0-1581bab2
# github.com:22 SSH-2.0-1581bab2
# github.com:22 SSH-2.0-1581bab2
Hi coded-by-49! You've successfully authenticated, but GitHub does not provide shell access.
/content/drive/MyDrive/Git_repos/Telecom-churn-model
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Tele_churn.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 143376c] Optimisation of Model precision and recall using feature engineering
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Tele_churn.ipynb (100%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 2.67 KiB | 143.00 KiB/s, don

In [6]:
from itertools import groupby
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

'''Data Preprocessing'''
from google.colab import drive
import urllib

# Importing file from googledrive
drive.mount('/content/drive')
tele_churn_csv = "/content/drive/MyDrive/GOOGLE COLLAB PROJECTS/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df_tele = pd.read_csv(tele_churn_csv)

# Cleaning out empty feature spots
df_tele["TotalCharges"] = df_tele["TotalCharges"].replace(" ", np.nan)
df_tele.dropna(subset = ["TotalCharges"], inplace=True)
df_tele["TotalCharges"] = pd.to_numeric(df_tele["TotalCharges"])

# Dropping unimportant features
df_tele.drop("customerID", axis = 1, inplace = True) # removed this unimportant feature

###### FEATURE ENGINNEERING
# Create the new column based on the two conditions
bins = [0,15,48,73]
labels = ["New_customers", "Established_customers", "Loyal_customers"]
df_tele['Tenure_Group'] = pd.cut(df_tele['tenure'], bins=bins, labels=labels, right=False)
df_tele['HighRisk_Interaction'] = ((df_tele['Contract'] == 'Month-to-month') & (df_tele['InternetService'] == 'Fiber optic')).astype(int)
df_tele['Tenure_MonthlyCharges'] = df_tele['tenure'] * df_tele['MonthlyCharges']

# Encoding our dataframe
encoded_df_tele = pd.get_dummies(df_tele, drop_first= True )
encoded_df_tele.head()

# Conversion of boolean columns into zeros and ones
bool_cols = encoded_df_tele.select_dtypes(include = "bool").columns
encoded_df_tele[bool_cols] = encoded_df_tele[bool_cols].astype(int)

###### FEATURE ENGINNEERING
encoded_df_tele['Fiber_Electronic'] = ((encoded_df_tele['InternetService_Fiber optic'] == 1) & (encoded_df_tele['PaymentMethod_Electronic check'] == 1)).astype(int)


# Conversion of tenure datatype numerical datatype
encoded_df_tele["tenure"] = encoded_df_tele["tenure"].astype(float)

# Storing all the numerical feature titles
all_numerical_features = encoded_df_tele.select_dtypes(include = np.number).columns.tolist()
needed_numerical_feature = [feature for feature in all_numerical_features if encoded_df_tele[feature].nunique()>2]


# Separation of future and labels
labels = encoded_df_tele["Churn_Yes"]
features = encoded_df_tele.drop("Churn_Yes", axis = 1)

# Creating our testing and training data
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size = 0.2, random_state = 42 )

# Scaling our non-categoical numerical features
Scaler = StandardScaler()
x_train[needed_numerical_feature] = Scaler.fit_transform(x_train[needed_numerical_feature])
x_test[needed_numerical_feature] = Scaler.transform(x_test[needed_numerical_feature])


"""IMPLEMENTATION OF LOGISTIC REGRESSION ALGORITHM FROM SCRATCH"""
################# THIS MODEL HAS BEEN TESTED
class custom_Log_regression:
  '''
      Attributes:
        learning_rate (float): Step size for gradient descent updates.
        n_iters (int): Maximum number of iterations for training.
  '''
  def __init__(self, learning_rate = 0.01, n_iters = 1000):
    self.learning_rate = learning_rate
    self.n_iters = n_iters

  def sigmoid(self,z):
    '''
       Sigmoid function, used for transformation of numerical prediction to 1's and 0's
       Args:
            z (numpy.ndarray): Input values (linear combination of features and weights).
       Our sigmoid then transfroms it into an array numpy.ndarray of probablilities between 0 and 1
    '''
    return 1/(1+np.exp(-z))

  def fit(self,X,Y):
    '''
    (Training function) Fit function would derive the weight and bias that
     occurs during the two minimum derivative of the loss function

    Args:
        X (numpy.ndarray): Training features (shape: [n_samples, n_features]).
        Y (numpy.ndarray): Training labels (shape: [n_samples]).
    '''
    self.num_samples, self.num_features = X.shape
    self.weights = np.zeros(self.num_features)
    self.bias = 0

    for _ in range(self.n_iters):
      y_num_prediction = np.dot(X, self.weights) + self.bias
      y_predicted = self.sigmoid(y_num_prediction)

      # Derivative of loss function
      dw = np.dot(X.T,(y_predicted - Y))/self.num_samples
      db = np.mean((y_predicted - Y))

      # Adjustment weights and bias after each iteration
      self.weights = self.weights - self.learning_rate * dw
      self.bias = self.bias - self.learning_rate * db

      # Stopping condition with eculidean normalisation
      if np.linalg.norm(db) < 0.001 and np.linalg.norm(dw) < 0.001:
        break # Minimum has been derived

  def score(self, x_test,y_test):
    '''
    Scoring function which tells us the confusion metrics (Accuracy) of our model
    '''

    # Deducing model prediction
    y_num_prediction = np.dot(x_test, self.weights) + self.bias
    self.y_predicted = self.sigmoid(y_num_prediction)

    # Addition of decision thresold to get standard decsion (1 or 0)
    self.y_predicted = np.where(self.y_predicted > 0.5, 1, 0)
    accuracy =  np.mean(self.y_predicted == y_test)
    return accuracy

# Importing algorithm
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(random_state=42)

# Resampling of Futures and Lables
x_train_bal, y_train_bal = adasyn.fit_resample(x_train,y_train)

'''TESTING OUT DATA SET TROUGHT ALGORITHMS '''
#note: Recall will be much important than precision for this model

#import models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
#importing scoring matrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score
#importing grid_test
from sklearn.model_selection import GridSearchCV



# train models and store them in a dictionary
possible_algorithms = {
    # "Logistic Regression": LogisticRegression(),
    # "XGBoost RF": XGBRFClassifier(n_estimators=100, random_state=42)
    # "LightGBM": LGBMClassifier(n_estimators=100, random_state=42)
}

# loop through the dictonary
# for alg_name,model in possible_algorithms.items():
#   print(f"{alg_name} \n")
#   f1 = cross_val_score(model, x_train_bal, y_train_bal, cv=5, scoring= 'f1').mean()
#   recall = cross_val_score(model, x_train_bal, y_train_bal, cv=5, scoring='recall').mean()
#   precision = cross_val_score(model, x_train_bal, y_train_bal, cv=5, scoring='precision').mean()

#   print(f"The f1 score for {alg_name} = {f1:.4f}")
#   print(f"The recall score for {alg_name} = {recall:.4f}")
#   print(f"The precision score for {alg_name} = {precision:.4f}\n")


"""STAND ALONE TEST ON RANDOMFORESTCLASSIFIER"""
#Hyperparameter tunning
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': [{0:1, 1:1.5}]  # Keep current class weight
}
grid_result =RandomizedSearchCV(RandomForestClassifier(random_state=42),param_grid,scoring="f1",cv=5,n_jobs=-1,random_state=42,)
grid_result.fit(x_train_bal,y_train_bal)
ideal_model = grid_result.best_estimator_

print(f"Best parameters: {grid_result.best_params_}")
print(f"Best Cross-Validation Recall: {grid_result.best_score_:.4f}")

y_prob = ideal_model.predict_proba(x_test)[:, 1]
#adjusting threshold
threshold = 0.4
y_pred = (y_prob >= threshold).astype(int)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Confusion Matrix: [TN={confusion_matrix(y_test, y_pred).ravel()[0]}, FP={confusion_matrix(y_test, y_pred).ravel()[1]}, FN={confusion_matrix(y_test, y_pred).ravel()[2]}, TP={confusion_matrix(y_test, y_pred).ravel()[3]}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1_score: {f1_score(y_test, y_pred):.4f}")


# bins = [0,15,48,73]
# labels = ["New_customers", "Established_customers", "Loyal_customers"]
# df_tele['Tenure_Group'] = pd.cut(df_tele['tenure'], bins=bins, labels=labels, right=False)
# df_tele['HighRisk_Interaction'] = ((df_tele['Contract'] == 'Month-to-month') & (df_tele['InternetService'] == 'Fiber optic')).astype(int)

# threshold = 0.4
# Best parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'class_weight': {0: 1, 1: 1.5}}
# Best Cross-Validation Recall: 0.8335
# Accuracy: 0.7257
# Confusion Matrix: [TN=736, FP=297, FN=89, TP=285
# Recall: 0.7620
# Precision: 0.4897
# F1_score: 0.5962

#threshold = 0.35
# Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# Best parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'class_weight': {0: 1, 1: 1.5}}
# Best Cross-Validation Recall: 0.8317
# Accuracy: 0.7129
# Confusion Matrix: [TN=700, FP=333, FN=71, TP=303
# Recall: 0.8102
# Precision: 0.4764
# F1_score: 0.6000

#threshold = 0.4 with engineered feature tenure_monthly_charges
# Best parameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'class_weight': {0: 1, 1: 1.5}}
# Best Cross-Validation Recall: 0.8325
# Accuracy: 0.7306
# Confusion Matrix: [TN=748, FP=285, FN=94, TP=280
# Recall: 0.7487
# Precision: 0.4956
# F1_score: 0.5964

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Best parameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'class_weight': {0: 1, 1: 1.5}}
Best Cross-Validation Recall: 0.8325
Accuracy: 0.7306
Confusion Matrix: [TN=748, FP=285, FN=94, TP=280
Recall: 0.7487
Precision: 0.4956
F1_score: 0.5964
